In [67]:
import openai
import dotenv
import networkx as nx

configenv = dotenv.dotenv_values(".env")
openai_api_key = configenv["OPENAI_API_KEY"]
openai.api_key = openai_api_key

In [68]:
seed_word = "molecular modelling"

In [69]:
prompt = "Give me a python list of 10 strings (in one single line bounded by '[' and ']' without newline characters) related to the string {}:".format(seed_word)

In [70]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  max_tokens=2000,
)

In [71]:
initial_generation = list(eval(response['choices'][0]['text'].strip(" \n")))

In [72]:
initial_generation

['Chemical structure',
 'Force fields',
 'Molecular dynamics',
 'Computational chemistry',
 'Energy minimization',
 'Protein-ligand interactions',
 'Quantum mechanics',
 'Molecular docking',
 'Simulations',
 'Biomolecular systems']

In [73]:
G = nx.DiGraph()
G.add_node(seed_word)
for w in initial_generation:
    G.add_node(w)
    G.add_edge(seed_word, w)

In [74]:
import copy

termsL = copy.deepcopy(initial_generation)
# TODO: add iter_depth

for j in initial_generation:
    generationsL = []

    prompt = "Give me a python list of 5 strings (in one single line bounded by '[' and ']' without newline characters) related to the string {} in the context of {}:".format(j, seed_word)
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2000,
    )
    try:
        generationsL = list(eval(response['choices'][0]['text'].strip(" \n")))
    except:
        continue

    for k in generationsL:
        if k not in termsL:
            termsL.append(k)
            G.add_node(k)
            G.add_edge(j, k)


In [75]:
from pyvis.network import Network

nt = Network(notebook=True, cdn_resources='remote')
nt.from_nx(G)
nt.show("KG_vis.html")

KG_vis.html
